In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
train_df = pd.read_csv(r'F:\TeraBoxDownload\train.csv')

print(train_df.columns)
print(train_df.info())
train_df.head()

Index(['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO', 'BHK_OR_RK',
       'SQUARE_FT', 'READY_TO_MOVE', 'RESALE', 'ADDRESS', 'LONGITUDE',
       'LATITUDE', 'TARGET(PRICE_IN_LACS)'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29451 non-null  object 
 1   UNDER_CONSTRUCTION     29451 non-null  int64  
 2   RERA                   29451 non-null  int64  
 3   BHK_NO                 29451 non-null  int64  
 4   BHK_OR_RK              29451 non-null  object 
 5   SQUARE_FT              29451 non-null  float64
 6   READY_TO_MOVE          29451 non-null  int64  
 7   RESALE                 29451 non-null  int64  
 8   ADDRESS                29451 non-null  object 
 9   LONGITUDE              29451 non-null  float64
 10  LATITUDE               29451 non-null  float64
 11 

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


> Observation: No missing values are in the dataset

---

### **EDA of the fetures**

> Observation: BHK_OR_RK and POSTED_BY are categorical variables

> BHK_OR_RK: 
>> We dont have enough training examples of the value 'RK'. So ignoring this categorical feature. So SQUARE_FT can be used to capture the information about the size of flat. For 'RK' the SQUARE_FT is less compared to 'BHK'


In [10]:
print(train_df['POSTED_BY'].unique())
print(train_df['BHK_OR_RK'].unique())

print("--- ---\n\nRecords for POSTED_BY: ")
print(train_df['POSTED_BY'].value_counts())

print("--- ---\n\nRecords for BHK_OR_RK: ")
print(train_df['BHK_OR_RK'].value_counts())

print("--- ---\n\nRecords for READY_TO_MOVE: ")
print(train_df['READY_TO_MOVE'].value_counts())


print("--- ---\n\nRecords for RESALE: ")
print(train_df['RESALE'].value_counts())

print("--- ---\n\nRecords for UNDER_CONSTRUCTION: ")
print(train_df['UNDER_CONSTRUCTION'].value_counts())

print("--- ---\n\nRecords for RERA: ")
print(train_df['RERA'].value_counts())

print("--- ---\n\nRecords for BHK_NO.: ")
print(train_df['BHK_NO'].value_counts())

['Owner' 'Dealer' 'Builder']
['BHK' 'RK']
--- ---

Records for POSTED_BY: 
Dealer     18291
Owner      10538
Builder      622
Name: POSTED_BY, dtype: int64
--- ---

Records for BHK_OR_RK: 
BHK    29427
RK        24
Name: BHK_OR_RK, dtype: int64
--- ---

Records for READY_TO_MOVE: 
1    24157
0     5294
Name: READY_TO_MOVE, dtype: int64
--- ---

Records for RESALE: 
1    27377
0     2074
Name: RESALE, dtype: int64
--- ---

Records for UNDER_CONSTRUCTION: 
0    24157
1     5294
Name: UNDER_CONSTRUCTION, dtype: int64
--- ---

Records for RERA: 
0    20088
1     9363
Name: RERA, dtype: int64
--- ---

Records for BHK_NO.: 
2     13324
3     10546
1      3574
4      1723
5       190
6        52
7        11
8        10
20        4
10        4
15        4
12        3
9         3
13        1
17        1
11        1
Name: BHK_NO, dtype: int64


> The features POSTED_BY, RESALE, RERA are under represented in given dataset. So while doing train_test_split stratification used while spliting to have equal distribution.


---

### **Feature Engineering**

> 1. New City feature derived from Address. City will fall in three category features
> 2. One-hot encoding for category feature POSTED_BY 


**1. City Feature**
* From address columns extract the city name.
* Then based on city name create three new features using one-hot-encoding whether the city falls under:
     * tier 1 city
     * tier 2 city
     * all other cities
     
**Reference:** https://en.wikipedia.org/wiki/Classification_of_Indian_cities

In [11]:
train_df['city'] = train_df['ADDRESS'].str.split(",").str[-1]
train_df.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),city
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0,Bangalore
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0,Mysore
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0,Bangalore
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5,Ghaziabad
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5,Kolkata


In [12]:
train_df.head(10)

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),city
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0,Bangalore
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0,Mysore
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0,Bangalore
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5,Ghaziabad
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5,Kolkata
5,Owner,0,0,3,BHK,1250.000000,1,1,"South Chittoor,Kochi",10.033280,76.282571,42.0,Kochi
6,Dealer,0,0,3,BHK,1495.053957,1,1,"Sodala,Jaipur",26.916347,75.795600,66.5,Jaipur
7,Owner,0,1,3,BHK,1181.012946,1,1,"Kharar,Mohali",30.740000,76.650000,52.0,Mohali
8,Dealer,0,1,2,BHK,1040.000000,1,1,"Bileshivale,Bangalore",13.054202,77.674002,41.6,Bangalore
9,Owner,0,1,2,BHK,879.120879,1,1,"Chromepet,Chennai",12.951610,80.140970,36.0,Chennai


In [13]:
def map_city(city):
    if city in ['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune', 'Maharashtra']:
        return 1
    elif city in ['Agra', 'Ajmer', 'Aligarh', 'Amravati', 'Amritsar', 'Asansol', 'Aurangabad', 'Bareilly', 
                  'Belgaum', 'Bhavnagar', 'Bhiwandi', 'Bhopal', 'Bhubaneswar', 'Bikaner', 'Bilaspur', 'Bokaro Steel City', 
                  'Chandigarh', 'Coimbatore', 'Cuttack', 'Dehradun', 'Dhanbad', 'Bhilai', 'Durgapur', 'Dindigul', 'Erode', 
                  'Faridabad', 'Firozabad', 'Ghaziabad', 'Gorakhpur', 'Gulbarga', 'Guntur', 'Gwalior', 'Gurgaon', 'Guwahati', 
                  'Hamirpur', 'Hubli–Dharwad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jammu', 'Jamnagar', 'Jamshedpur', 
                  'Jhansi', 'Jodhpur', 'Kakinada', 'Kannur', 'Kanpur', 'Karnal', 'Kochi', 'Kolhapur', 'Kollam', 'Kozhikode', 
                  'Kurnool', 'Ludhiana', 'Lucknow', 'Madurai', 'Malappuram', 'Mathura', 'Mangalore', 'Meerut', 'Moradabad', 
                  'Mysore', 'Nagpur', 'Nanded', 'Nashik', 'Nellore', 'Noida', 'Patna', 'Pondicherry', 'Purulia', 'Prayagraj', 
                  'Raipur', 'Rajkot', 'Rajahmundry', 'Ranchi', 'Rourkela', 'Ratlam', 'Salem', 'Sangli', 'Shimla', 'Siliguri', 
                  'Solapur', 'Srinagar', 'Surat', 'Thanjavur', 'Thiruvananthapuram', 'Thrissur', 'Tiruchirappalli', 'Tirunelveli', 
                  'Tiruvannamalai', 'Ujjain', 'Bijapur', 'Vadodara', 'Varanasi', 'Vasai-Virar City', 'Vijayawada', 'Visakhapatnam', 
                  'Vellore', 'Warangal']:
        return 2
    else:
        return 3
    
train_df['city_tier'] = train_df['city'].apply(map_city)

In [14]:
train_df.head(5)

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),city,city_tier
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0,Bangalore,1
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0,Mysore,2
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0,Bangalore,1
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5,Ghaziabad,2
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5,Kolkata,1


**2. Posted_by Feature**
* One-hot encoding of posted_by category feature

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[("posted_by_transform", OneHotEncoder(), ['POSTED_BY'])], remainder='passthrough')

train_df_tranformed = pd.DataFrame(transformer.fit_transform(train_df))

# Set Column names in for the transformed Dataframe
train_df_tranformed.columns = ['POSTED_BY_builder', 'POSTED_BY_dealer', 'POSTED_BY_Owner','UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK', 'SQUARE_FT', 
                               'READY_TO_MOVE', 'RESALE', 'ADDRESS', 'LONGITUDE', 'LATITUDE', 'TARGET(PRICE_IN_LACS)', 'city', 'city_tier']

train_df_tranformed.head(5)

,POSTED_BY_builder,POSTED_BY_dealer,POSTED_BY_Owner,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),city,city_tier
0,0.0,0.0,1.0,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.96991,77.59796,55.0,Bangalore,1
1,0.0,1.0,0.0,0,0,2,BHK,1275.0,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0,Mysore,2
2,0.0,0.0,1.0,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0,Bangalore,1
3,0.0,0.0,1.0,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.6423,77.3445,62.5,Ghaziabad,2
4,0.0,1.0,0.0,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.5922,88.484911,60.5,Kolkata,1


---

### **Explore Models - VERSION 1**

In [17]:
selected_features = ['UNDER_CONSTRUCTION', 'RERA', 'BHK_NO', 'SQUARE_FT', 'READY_TO_MOVE', 'RESALE', 'TARGET(PRICE_IN_LACS)', 'city_tier']

#selected_features = ['BHK_NO.', 'RESALE','SQUARE_FT', 'TARGET(PRICE_IN_LACS)']

df_features_selected = train_df[selected_features]
df_features_selected.head()


,UNDER_CONSTRUCTION,RERA,BHK_NO,SQUARE_FT,READY_TO_MOVE,RESALE,TARGET(PRICE_IN_LACS),city_tier
0,0,0,2,1300.236407,1,1,55.0,1
1,0,0,2,1275.000000,1,1,51.0,2
2,0,0,2,933.159722,1,1,43.0,1
3,0,1,2,929.921143,1,1,62.5,2
4,1,0,2,999.009247,0,1,60.5,1


In [18]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_features_selected,test_size=0.20, random_state=0)
#train, test = train_test_split(df_features_selected,test_size=0.20, random_state=0, stratify=df_features_selected[['READY_TO_MOVE', 'RESALE', 'UNDER_CONSTRUCTION', 'RERA']])

X_train = train[train.columns.difference(['TARGET(PRICE_IN_LACS)'])]
y_train = train['TARGET(PRICE_IN_LACS)']

X_test = test[test.columns.difference(['TARGET(PRICE_IN_LACS)'])]
y_test = test['TARGET(PRICE_IN_LACS)']

print(X_train.shape)
print(X_test.shape)

(23560, 7)
(5891, 7)


In [19]:
X_test.head(5)

,BHK_NO,READY_TO_MOVE,RERA,RESALE,SQUARE_FT,UNDER_CONSTRUCTION,city_tier
26350,3,1,0,1,1316.135825,0,2
23308,2,1,0,1,1080.201243,0,2
16028,2,1,0,1,1183.081928,0,3
28605,2,0,1,0,728.792046,1,3
21083,4,1,1,1,3563.437474,0,1


In [20]:
# Linear Regression

from sklearn.linear_model import LinearRegression

lin_reg_model = LinearRegression()

lin_reg_model.fit(X_train, y_train)

print('KNN Regressor: ', lin_reg_model.score(X_test, y_test))

KNN Regressor:  0.35857713938315927


In [21]:
# KNN Regressor

from sklearn.neighbors import KNeighborsRegressor

knn_reg_model = KNeighborsRegressor(n_neighbors=5)

knn_reg_model.fit(X_train, y_train)

print('KNN Regressor: ', knn_reg_model.score(X_test, y_test))

KNN Regressor:  0.7005988163354708


In [22]:
# Random Forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

preds = rf_model.predict(X_test)

print('Random Forest: ', r2_score(y_test, preds))

Random Forest:  0.9178271711014228


In [23]:
# XG Boost

from xgboost import XGBRegressor

xgboost_model = XGBRegressor(n_estimators=1000, learning_rate=0.1, random_state=42)

xgboost_model.fit(X_train, y_train)

preds = xgboost_model.predict(X_test)



---

### Prediction for Test dataset - Version 1

In [24]:
test_df = pd.read_csv(r'F:\TeraBoxDownload\train.csv')

# Feature Engineering
# 1. Derive city tier feature
train_df['city_tier'] = train_df['city'].apply(map_city)

# 2. One-hot encoding for POSTED_BY
test_df_tranformed = pd.DataFrame(transformer.fit_transform(test_df))

# Set Column names in for the transformed Dataframe
train_df_tranformed.columns = ['POSTED_BY_builder', 'POSTED_BY_dealer', 'POSTED_BY_Owner','UNDER_CONSTRUCTION', 'RERA', 'BHK_NO', 'BHK_OR_RK', 'SQUARE_FT', 
                               'READY_TO_MOVE', 'RESALE', 'ADDRESS', 'LONGITUDE', 'LATITUDE', 'TARGET(PRICE_IN_LACS)', 'city', 'city_tier']

# Making Prediction
# Using the trained XG Boost model
selected_features = ['BHK_NO','READY_TO_MOVE', 'RERA', 'RESALE', 'SQUARE_FT', 'UNDER_CONSTRUCTION','city_tier']

X_test = train_df[selected_features]
for feature in selected_features:
    X_test[feature] =  pd.to_numeric(X_test[feature])

df_test_output = xgboost_model.predict(X_test)

# Save to submission.csv file
df_output = pd.DataFrame({'Id': X_test.index, 'PredictedSalePrice': df_test_output})
df_output.to_csv('submission.csv', index=False)
df_output.head()

C:\Users\ychau\AppData\Local\Temp\ipykernel_8848\4288110788.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[feature] =  pd.to_numeric(X_test[feature])


,Id,PredictedSalePrice
0,0,70.207230
1,1,48.908886
2,2,63.275204
3,3,35.734955
4,4,70.569489


In [25]:
import pickle
pickle.dump(xgboost_model,open('data.pkl','wb'))